In [1]:
from dotenv import load_dotenv

load_dotenv()


True

#### State 설정

In [2]:
from typing import Literal
from typing_extensions import TypedDict
from langchain_openai import ChatOpenAI

class AgentState(TypedDict):
    """The state of our agent."""
    question: str
    certainty_score: int
    search_results: list
    web_score: str
    repo_name: str
    generation: str

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

